<a href="https://colab.research.google.com/github/adenilso/sucupira-indicadores/blob/master/(Working)_Sucupira_Indicadores_Quadrienal_2017_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import io
from plotnine import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/CAPES2018-2021/Quadrienal 2017-2020/Indicadores/'
sheetnames = ['Sumário']
sheets = dict([[sn, pd.read_csv(path + 'CC-' + sn + '.csv')] for sn in sheetnames])
sumario = sheets['Sumário']

In [ ]:
# estratos depende do sufixo - sem sufixo, primeiro A1 (periódicos), .1 (conferências) ...
estratosbase = ["B4", "B3", "B2", "B1", "A4", "A3", "A2", "A1"]
sufixo = ".1"
pesosbase = dict({"B4": 0.05,
               "B3": 0.10,
               "B2": 0.20,
               "B1": 0.50,
               "A4": 0.625,
               "A3": 0.75, 
               "A2": 0.875,
               "A1": 1.0
              })
estratos = [e + sufixo for e in estratosbase]
pesos = dict([[e + sufixo, pesosbase[e]] for e in estratosbase])

In [ ]:
sumario_acad = sumario[pd.notnull(sumario['Nome'])]
sumario_acad = sumario_acad[sumario_acad["Modalidade"] == "ACADEMICO"]

In [ ]:
qualis_per = pd.melt(sumario_acad, id_vars=["Sigla", "Média Docentes"], value_vars=estratos, var_name="Estrato", value_name="Qte")
qualis_per = qualis_per[pd.notnull(qualis_per["Sigla"])]
qualis_per = qualis_per[qualis_per["Estrato"].isin(estratos)]
Estrato_cat = pd.Categorical(qualis_per["Estrato"], categories=estratos)
qualis_per = qualis_per.assign(Estrato = Estrato_cat)
ponderamento = lambda e: e["Qte"] * pesos[e["Estrato"]] / e["Média Docentes"] /
qualis_per = qualis_per.assign(QtePeso = qualis_per.apply(ponderamento, axis=1))

SyntaxError: ignored

In [ ]:
order = qualis_per[["Sigla", "QtePeso"]].groupby(by="Sigla")["QtePeso"].sum().sort_values().keys().tolist()
order.reverse()

In [ ]:
Sigla_cat = pd.Categorical(qualis_per["Sigla"], categories=order)
qualis_per = qualis_per.assign(Sigla_cat = Sigla_cat)

In [ ]:
(
    ggplot(qualis_per)
    + aes(x="Sigla_cat", y="QtePeso", fill="Estrato")
    + geom_bar(stat="identity", alpha=1, position=position_stack())
    + theme(axis_text_x=element_text(angle=90, size=5))
    + theme(figure_size=(12.0, 12.0))
    + labs(x="Programas", y="Quantidade", legend="Estratos")
    + ggtitle("Publicações Qualis ("+sufixo+") Ponderado!")
)

In [ ]:
def plotPublicacaoQualis(sufixo = "", titulo="Qualis Per", ponderamento = lambda e: e["Qte"] * pesos[e["Estrato"]] / e['Média Docentes']):
  estratosbase = ["B4", "B3", "B2", "B1", "A4", "A3", "A2", "A1"]
  sufixo = ""
  pesosbase = dict({"B4": 0.05,
               "B3": 0.10,
               "B2": 0.20,
               "B1": 0.50,
               "A4": 0.625,
               "A3": 0.75, 
               "A2": 0.875,
               "A1": 1.0
              })
  estratos = [e + sufixo for e in estratosbase]
  pesos = dict([[e + sufixo, pesosbase[e]] for e in estratosbase])
  sumario_acad = sumario[pd.notnull(sumario['Nome'])]
  sumario_acad = sumario_acad[sumario_acad["Modalidade"] == "ACADEMICO"]
  qualis_per = pd.melt(sumario_acad, id_vars=["Sigla", "Média Docentes"], value_vars=estratos, var_name="Estrato", value_name="Qte")
  qualis_per = qualis_per[pd.notnull(qualis_per["Sigla"])]
  qualis_per = qualis_per[qualis_per["Estrato"].isin(estratos)]
  Estrato_cat = pd.Categorical(qualis_per["Estrato"], categories=estratos)
  qualis_per = qualis_per.assign(Estrato = Estrato_cat)
  #ponderamento = lambda e: e["Qte"] * pesos[e["Estrato"]] / e['Média Docentes']
  qualis_per = qualis_per.assign(QtePeso = qualis_per.apply(ponderamento, axis=1))
  order = qualis_per[["Sigla", "QtePeso"]].groupby(by="Sigla")["QtePeso"].sum().sort_values().keys().tolist()
  order.reverse()
  Sigla_cat = pd.Categorical(qualis_per["Sigla"], categories=order)
  qualis_per = qualis_per.assign(Sigla_cat = Sigla_cat)
  gg = (
    ggplot(qualis_per)
    + aes(x="Sigla_cat", y="QtePeso", fill="Estrato")
    + geom_bar(stat="identity", alpha=1, position=position_stack())
    + theme(axis_text_x=element_text(angle=90, size=5))
    + theme(figure_size=(10.0, 10.0))
    + labs(x="Programas", y="Quantidade", legend="Estratos")
    + ggtitle(titulo)
  )
  return gg

In [ ]:
#plotPublicacaoQualis("", "Periódicos", lambda e: e["Qte"] * pesos[e["Estrato"]] / e['Média Docentes'])
plotPublicacaoQualis("", "Periódicos", lambda e: e["Qte"] * pesos[e["Estrato"]])